# Met2 Dataset

japanese encoding : https://en.wikipedia.org/wiki/JIS_encoding



In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader



In [ ]:
import gzip 

met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    file_content = f.read()
document = str(file_content, 'EUC-JP')


In [ ]:
import gzip

from bs4 import BeautifulSoup

def get_samples(document):
    b_sentence, b_label = [], []

    soup = BeautifulSoup(document, 'html.parser')
    doc_list = list(soup.find_all('doc'))

    for doc in doc_list:
        paragraphs = list(doc.find_all('p'))
        for paragraph in paragraphs:
            p = paragraph

            # labeling
            cursor, sentence, label = 0, [], []

            tag_list = list(p.find_all(['enamex','timex','numex']))
            for tag in tag_list:
                cursor_tag = p.text.find(tag.text, cursor, len(p.text))

                if cursor_tag-cursor>0:
                    sentence.append(p.text[cursor:cursor_tag].replace('\u3000','').replace('\n',''))
                    label.append('None')
                sentence.append(p.text[cursor_tag:cursor_tag+len(tag.text)])
                label.append(tag['type'])

                cursor = cursor_tag+len(tag.text)

            ## paragraph太長了，sentence超過5 當作下一筆sample
            for i in range(0,len(sentence)-5,5):
                b_sentence.append(sentence[i:i+5])
                b_label.append(label[i:i+5])

    return b_sentence, b_label

met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    file_content = f.read()
document = str(file_content, 'EUC-JP')

b_sentence, b_label = get_samples(document)

In [ ]:
len(b_sentence), len(b_label)

In [ ]:
idx = 859
b_sentence[idx], b_label[idx]

## get tags

In [ ]:

def get_tags(document):
    tags = set()
    soup = BeautifulSoup(document, 'html.parser')
    name_tags = soup.find_all(['enamex','timex','numex'])
    for tag in name_tags:
        tags.update([tag['type']])
    return ['None']+sorted(list(tags))

met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    document = str(f.read(), 'EUC-JP')
    
tags = get_tags(document)
tags

## get samples

In [ ]:
met2_path = '/media/D/ADL2020-SPRING/project/met2/met2data/training/japanese/keys/keys.970827.gz'

with gzip.open(met2_path, 'rb') as f:
    document = str(f.read(), 'EUC-JP')
    
soup = BeautifulSoup(document, 'html.parser')

doc_list = list(soup.find_all('doc'))
#random.shuffle(doc_list)

for doc in doc_list:
    para = doc.find_all('p')
    #print(len(para),para)
    input("")
    for p in para:
        print(p.text)
        data.append(p.text)
        input("")


In [ ]:

para = para.find_next('enamex')
para['type'], para.text


In [ ]:
para = para.next_sibling
para

# Cinnamon Dataset

## inference

In [2]:
from ..dataset import *
from train import *

import torch.nn.functional as F

import os, warnings, argparse
warnings.filterwarnings('ignore')

def parse_args(string=None):
    parser = argparse.ArgumentParser()
    parser.add_argument('--lr', default=2e-5,
                        type=float, help='leanring rate')
    parser.add_argument('--epoch', default=5,
                        type=int, help='epochs')
    parser.add_argument('--batch-size', default=4,
                        type=int, help='batch size')
    parser.add_argument('--gpu', default="1",
                        type=str, help="0:1080ti 1:1070")
    parser.add_argument('--num-workers', default=8,
                        type=int, help='dataloader num workers')
    parser.add_argument('--cinnamon-data-path', default='/media/D/ADL2020-SPRING/project/cinnamon/',
                        type=str, help='cinnamon dataset')
    parser.add_argument('--load-model', default='./naive_baseline/ckpt/epoch_6.pt',
                        type=str, help='.pt model file ')
    parser.add_argument('--save-result-path', default='./naive_baseline/result/',
                        type=str, help='.pt model file save dir')
    
    args = parser.parse_args() if string is None else parser.parse_args(string)
    if not os.path.exists(args.save_result_path): os.makedirs(args.save_result_path)
    return args

def inference(args, dataloader):
    model = Model()

if __name__ == '__main__':
    args = parse_args('')

    os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu 
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

    tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=True)

    train_dataset = Cinnamon_Dataset(f'{args.cinnamon_data_path}/train/', tokenizer)
    valid_dataset = Cinnamon_Dataset(f'{args.cinnamon_data_path}/dev/', tokenizer)
    valid_dataloader = DataLoader(valid_dataset,
                                     batch_size = 1,
                                     num_workers = args.num_workers,
                                     collate_fn = valid_dataset.collate_fn)

model = Model()
model.load('ckpt/epoch_7.pt').cuda()
# 0
print(0)

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.1.1) doesn't match a supported version!
  RequestsDependencyWarning)


AttributeError: 'Namespace' object has no attribute 'save_path'

In [ ]:
tags = train_dataset.tags
tags

In [ ]:
model.eval()
with torch.no_grad():
    for idx, (_input, _label) in enumerate(valid_dataloader):
        b = _input.shape[0]
        
        _predict = model(_input.cuda())
        
        assert(len(_predict[0])==len(_input[0]))
        # sentence
        #print(list(_input[0].cpu().numpy()))
        sentence = np.array([tokenizer.convert_ids_to_tokens([_id])[0] for _id in list(_input[0].cpu().numpy())])
        # tag
        sentence_tag = (_predict[0]>0.5).cpu().numpy()
        
        #print(sentence,sentence_tag)
        
        for j,tag in enumerate(tags):
            show = sentence[sentence_tag[:,j]]
            if len(show)==0:
                continue
            print(tags[j])
            print(show)
        input("")